In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
main_dir = r'../../..'
data_path = main_dir+'/data'
import sys
sys.path.append(main_dir)
from BayDS.lib.pipeline import *

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



In [ ]:
from typing import List, Set, Dict, Optional, Any, Tuple, Type, Union
from BayDS.lib.io import *
import os


In [ ]:
data_dir =f'{main_dir}/Snapshots/99'
input_dir = f'{main_dir}/Data'

p = Pipeline(working_folder=f'{main_dir}/Snapshots/1/Analyses')


# p.add_node(IEEEFraudTransactionLoaderNode, None, 'transactions',
#            params={
#                'input_directory': input_dir
#            })

p.add_node(LoaderNode, None, 'data',
           params={
               'input_directory': data_dir,
               'file': 'data.pkl'
           })
# p.add_node(LoaderNode, None, 'numerical_columns',
#            params={
#                'input_directory': data_dir,
#                'file': 'numerical_columns.yaml'
#            })
# p.add_node(LoaderNode, None, 'categorical_columns',
#            params={
#                'input_directory': data_dir,
#                'file': 'categorical_columns.yaml'
#            })

p.run(verbose=True)


In [ ]:
df = p.data['data']
df.card_id.value_counts()

In [ ]:
from datetime import datetime,timedelta
from typing import NamedTuple, List, Dict,Set
class User(object):
    user_id:int
    card_id:str
    start_date:datetime
    last_transaction_date:datetime
    transaction_ids: List[str]
    def __init__(self,**kwargs):
        for k,v in kwargs.items():
            setattr(self,k,v)
    def __repr__(self):
        return str(self.__dict__)
users = []
users_by_card_id_by_first_date:Dict[int,Dict[datetime,User]] = {}

START_DATE = '1900-12-01'
nan_startdate = datetime.strptime(START_DATE, "%Y-%m-%d")

START_DATE = '1950-12-01'
zero_startdate = datetime.strptime(START_DATE, "%Y-%m-%d")


In [ ]:
percentage = 0
for card_id in range(0,19213):
    new_percentage = card_id*100//19213
    if new_percentage > percentage:
        percentage = new_percentage
        if percentage % 5 ==0:
            print(f'{percentage}%')
    user_df = df[df.card_id==card_id]
    D_cols = ['D%d'%i for i in range(1,5)]
    filtered = user_df[['Date']+D_cols]
#     filtered['diff'] = filtered.Date.diff()
    for index,row in filtered.iterrows():
        dt = row['Date'].date()
        if pd.isnull(row['D1']):
            first_transaction_date = nan_startdate
            days_since_first_transaction = (dt - startdate.date()).days
        else:
            days_since_first_transaction = int(row['D1'])
            if days_since_first_transaction == 0:
                first_transaction_date = zero_startdate
            else:
                first_transaction_date = dt-timedelta(days=days_since_first_transaction)
        create_new = False
        if card_id not in users_by_card_id_by_first_date:
            users_by_card_id_by_first_date[card_id] = {}
            create_new = True
        else:
            if first_transaction_date not in users_by_card_id_by_first_date[card_id]:
                create_new = True
            else:
                u = users_by_card_id_by_first_date[card_id][first_transaction_date]
                u.last_transaction_date = dt
                u.transaction_ids.append(index)
        if create_new:
            u = User(user_id=len(users),
                     card_id=card_id,
                     start_date=first_transaction_date,
                     last_transaction_date=dt,
                     transaction_ids=[index])
            users.append(u)
            users_by_card_id_by_first_date[card_id][first_transaction_date] = u
    


In [ ]:
filtered


In [ ]:
card_id = 0
users_by_card_id_by_first_date[card_id].keys()
x = list(users_by_card_id_by_first_date[card_id].keys())
y = [len(users_by_card_id_by_first_date[card_id][k].transaction_ids) for k in x]


In [ ]:
df[df.card_id == 6006][['isFraud','Date']+D_cols]

In [ ]:
l = len(users)


In [ ]:
# df_new_card_id = pd.DataFrame(index=df.index)
# df_new_card_id['new_card_id'] = -1
# df_new_card_id['new_card_id'].astype(np.int32)
# df_start_date = pd.DataFrame(index=df.index)
# df_start_date['start_date'] = -1
# df_start_date['start_date'].astype(np.int32)

index_to_new_card_id = {}
index_to_new_start_date = {}
percentage = 0
for i,u in enumerate(users):
    new_percentage = i*100//l
    if new_percentage > percentage:
        percentage = new_percentage
        if percentage % 5 ==0:
            print(f'{percentage}%')
#     df_new_card_id.loc[u.transaction_ids,'new_cardId'] = u.user_id
#     df_start_date.loc[u.transaction_ids,'start_date'] = u.start_date
    for idx in u.transaction_ids:
        index_to_new_card_id[idx] = u.user_id
        index_to_new_start_date[idx] = u.start_date
            


In [ ]:
new_card_id = np.zeros(len(df.index),dtype=np.int32)
start_date = np.zeros(len(df.index),dtype='datetime64[ms]')
for i,idx in enumerate(df.index):
    if idx not in index_to_new_card_id:
        print(f'Not Found {idx}')
        continue
    new_card_id[i] = index_to_new_card_id[idx]
    start_date[i] = index_to_new_start_date[idx]
    

In [ ]:
df['new_card_id'] = new_card_id

In [ ]:
df['start_date'] = start_date

In [ ]:
df[['new_card_id','start_date']].to_pickle(f'{p.working_folder}/new_id.pkl')